In [ ]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('project2.ok')

# Project 2: Natural Experiments: The Vietnam War Draft

### Logistics


**Deadline.** This project is due at 11:59pm P.T. on Friday, 4/9. Projects will be accepted up to 2 days (48 hours) late; a project submitted less than 24 hours after the deadline will receive 2/3 credit, a project submitted between 24 and 48 hours after the deadline will receive 1/3 credit, and a project submitted 48 hours or more after the deadline will receive no credit. It's **much** better to be early than late, so start working now.

**Checkpoint.** For full credit on the checkpoint, you must complete section 1 and 2 of Part 1, pass all public autograder tests, and submit to okpy by 11:59pm P.T. on Friday, 4/2. This is worth 1 point of your entire project grade. After you've submitted the checkpoint, you may still change your answers before the project deadline - only your final submission will be graded for correctness. You will have some lab time to work on these questions, but we recommend that you start the project before lab and leave time to finish the checkpoint afterward.

**Partners.** You may work with one other partner; your partner must be from your assigned lab section. Only one of you is required to submit the project. On [okpy.org](http://okpy.org), the person who submits should also designate their partner so that both of you receive credit. Feel free to split up the work between your partner, share what you complete with each other in a google document, and share your screen in a Zoom call as you work together.

**Rules.** Don't share your code with anybody but your partner. You are welcome to discuss questions with other students, but don't share the answers. The experience of solving the problems in this project will prepare you for exams (and life). If someone asks you for the answer, resist! Instead, you can demonstrate how you would solve a similar problem.

**Support.** You are not alone! Come to office hours, post on Piazza, and talk to your classmates. If you want to ask about the details of your solution to a problem, make a private Piazza post and the staff will respond. If you're ever feeling overwhelmed or don't know how to make progress, email your discussion TA or tutor for help. You can find contact information for the staff on the [course website](http://data8.org/sp21/staff.html).

**Tests.** The tests that are given are **not comprehensive** and passing the tests for a question **does not** mean that you answered the question correctly. Tests usually only check that your table has the correct column labels. However, more tests will be applied to verify the correctness of your submission in order to assign your final score, so be careful and check your work! You might want to create your own checks along the way to see if your answers make sense. Additionally, before you submit, make sure that none of your cells take a very long time to run (several minutes). Each question is worth 1 point of your total project grade.

**Free Response Questions:** Make sure that you put the answers to the written questions in the indicated cell we provide. **Every free response question should include an explanation** that adequately answers the question. Check to make sure that you have a [Gradescope](http://gradescope.com) account, which is where the scores to the free response questions will be posted. If you do not, make sure to reach out to your assigned TA.

**Advice.** Develop your answers incrementally. To perform a complicated table manipulation, break it up into steps, perform each step on a different line, give a new name to each result, and check that each intermediate result is what you expect. You can add any additional names or functions you want to the provided cells. Make sure that you are using distinct and meaningful variable names throughout the notebook. Along that line, **DO NOT** reuse the variable names that we use when we grade your answers.

You **never** have to use just one line in this project or any others. Use intermediate variables and multiple lines as much as you would like!

All of the concepts necessary for this project are found in the textbook. If you are stuck on a particular problem, reading through the relevant textbook section often will help clarify the concept.

To get started, load `datascience`, `numpy`, `plots`, and `ok`.

In [1]:
# Don't change this cell; just run it. 
# When you log-in please hit return (not shift + return) after typing in your email

from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
np.set_printoptions(legacy='1.13')

from client.api.notebook import *
def new_save_notebook(self):
    """ Saves the current notebook by
        injecting JavaScript to save to .ipynb file.
    """
    try:
        from IPython.display import display, Javascript
    except ImportError:
        log.warning("Could not import IPython Display Function")
        print("Make sure to save your notebook before sending it to OK!")
        return

    if self.mode == "jupyter":
        display(Javascript('IPython.notebook.save_checkpoint();'))
        display(Javascript('IPython.notebook.save_notebook();'))
    elif self.mode == "jupyterlab":
        display(Javascript('document.querySelector(\'[data-command="docmanager:save"]\').click();'))   

    print('Saving notebook...', end=' ')

    ipynbs = [path for path in self.assignment.src
              if os.path.splitext(path)[1] == '.ipynb']
    # Wait for first .ipynb to save
    if ipynbs:
        if wait_for_save(ipynbs[0]):
            print("Saved '{}'.".format(ipynbs[0]))
        else:
            log.warning("Timed out waiting for IPython save")
            print("Could not automatically save \'{}\'".format(ipynbs[0]))
            print("Make sure your notebook"
                  " is correctly named and saved before submitting to OK!".format(ipynbs[0]))
            return False                
    else:
        print("No valid file sources found")
    return True

def wait_for_save(filename, timeout=600):
    """Waits for FILENAME to update, waiting up to TIMEOUT seconds.
    Returns True if a save was detected, and False otherwise.
    """
    modification_time = os.path.getmtime(filename)
    start_time = time.time()
    while time.time() < start_time + timeout:
        if (os.path.getmtime(filename) > modification_time and
            os.path.getsize(filename) > 0):
            return True
        time.sleep(0.2)
    print("\nERROR!\n YOUR SUBMISSION DID NOT GO THROUGH. PLEASE TRY AGAIN. IF THIS PROBLEM PERSISTS POST ON PIAZZA RIGHT AWAY.\n ERROR!" + "\n"*20)
    return False

Notebook.save_notebook = new_save_notebook

from client.api.notebook import Notebook
ok = Notebook('project2.ok')
ok.auth(force=True)
_ = ok.submit()

# Background
"The Vietnam War (Vietnamese: Chiến tranh Việt Nam), also known as the Second Indochina War, and in Vietnam as the Resistance War Against America (Vietnamese: Kháng chiến chống Mỹ) or simply the American War, was a conflict in Vietnam, Laos, and Cambodia from 1 November 1955 to the fall of Saigon on 30 April 1975." - Wikipedia.

The Vietnam War was the last war in which the United States employed a policy of conscription—"the compulsory induction of individuals into the Armed Services". Conscription means you are forced to join the military under certain legal penalties. 

Men are chosen to be conscripted by the "draft".

This assignment is in part inspired by an assignment from Professor Yun S. Song's STAT 135 class.

# Part 1: The 1969 Draft Lottery
The first draft of the Vietnam War took place on December 1st, 1969. Men aged 19-26, born in 1944-1950, were eligible to be drafted. 366 plastic lottery balls, representing every day of the year, and the leap day February 29th, were placed into an urn and selected one at a time and the order was recorded. For the 1969 lottery, all men with birthdays whose corresponding lottery balls were selected 195th or lower were drafted\*.

\*drafted in this case means "classified as available for military service, were called to report for possible induction."

Run the following cell to load in the `lotto` table.

The `lotto` table was sourced from [randomservices.org](https://www.randomservices.org/random/data/Draft.html).

In [2]:
# Read in the data
lotto = Table.read_table("lottery.tsv", sep="\t")
lotto

The `lotto` table contains the columns `M` and `D` corresponding to the month and day of the birthdays on the lottery balls, and a column for each draft that contains the order that the birthday was drawn in each of the four draft lotteries. For this section we are only interested in the \`69 draft which would be the `N69` column.

## Section 1: Investigating December
Let's check to see if the 1969 lottery process was truly random and fair.

The `lotto69` table has only the columns from the `lotto` table pertinent to the 1969 draft.

In [3]:
lotto69 = lotto.select("M", "D", "N69")

**Question 1.1.1:** Let's investigate the data visually first. Draw a scatter plot of draft number vs month.

*Note: although month is a categorical value we are using it as a numerical value for the sake of this plot.*

<!--
BEGIN QUESTION
name: q1_1_1
manual: true
-->
<!-- EXPORT TO PDF -->

In [4]:
...

**Question 1.1.2:** It's a bit hard to tell what's happening in that plot. Aggregate the data and plot a bar chart of the average draft number vs month.

<!--
BEGIN QUESTION
name: q1_1_2
manual: true
-->
<!-- EXPORT TO PDF -->

In [5]:
...

It seems that December (month 12) has a lower average draft number than the other months.

**Question 1.1.3:** How many birthdays in December were chosen to be drafted? Recall that if your lottery number was less than or equal to 195, you would be drafted.

<!--
BEGIN QUESTION
name: q1_1_3
-->

In [6]:
num_drafted = ...
num_drafted

In [ ]:
ok.grade("q1_1_3");

We will perform a hypothesis test to check if this high number of birthdays selected in December, 1969, is statistically significant: was the lottery truly random?

**Question 1.1.4:** What is our null hypothesis?
<!--
BEGIN QUESTION
name: q1_1_4
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

**Question 1.1.5:** What is our alternative hypothesis?
<!--
BEGIN QUESTION
name: q1_1_5
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

The test statistic we will use is the number of birthdays selected in December.

**Question 1.1.6:** Fill out the function `statistic` that returns one value of our test statistic given an array of draft numbers for the 31 days in December.

<!--
BEGIN QUESTION
name: q1_1_6
-->

In [9]:
def statistic(dec_array):
    ...

obs_test_stat = ...
obs_test_stat

In [ ]:
ok.grade("q1_1_6");

**Question 1.1.7:** Fill out the function `simulate` to simulate one sample under the null hypothesis and return a value of our test statistic. 

*Hint 1: Draft numbers start at 1!*

*Hint 2: Remember that leap year birthdays are included!*

<!--
BEGIN QUESTION
name: q1_1_7
-->

In [12]:
# These should be useful!
num_days_dec = 31 
possible_draft_numbers = ...

def simulate():
    sample_dec_array = ...
    ...

simulate()

In [ ]:
ok.grade("q1_1_7");

**Question 1.1.8:** Simulate and store 10000 values of the test statistic in the array `test_stats` and draw a histogram of the values.

<!--
BEGIN QUESTION
name: q1_1_8
manual: true
-->
<!-- EXPORT TO PDF -->

In [16]:
test_stats = ...
reps = ...

...

Table().with_columns("Simulated Statistics", test_stats).hist("Simulated Statistics")

**Question 1.1.9:** Compute the p-value of the hypothesis test and assign it to `dec_p_val`.

<!--
BEGIN QUESTION
name: q1_1_9
-->

In [17]:
dec_p_val = ...
dec_p_val

In [ ]:
ok.grade("q1_1_9");

**Question 1.1.10:** According to the [New York Times](https://www.nytimes.com/1970/01/04/archives/statisticians-charge-draft-lottery-was-not-random.html), 
> "draft officials acknowledge, the capsules with the later months were not mixed as thoroughly as those of the earlier months." 

Using a p-value cutoff of 5%, assign `reject_dec` to a boolean expression that evaluates to `True` if we should reject the null hypothesis and `False` if we fail to.

<!--
BEGIN QUESTION
name: q1_1_10
-->

In [20]:
reject_dec = ...
reject_dec

In [ ]:
ok.grade("q1_1_10");

## Section 2: The Other Months

It appears that December had some issues in the lottery, but what about the other months? For each month let's test a slightly different hypothesis, not that the number drafted in that month is too high, but that it is either too high or too low.

**Question 1.2.1:** Define the function `new_stat` that computes the value for our new test statistic given `prop_selected`, the proportion of days in the month selected in a sample. Our new test statistic is the absolute difference between the sample proportion selected and the expected proportion selected.

<!--
BEGIN QUESTION
name: q1_2_1
-->

In [23]:
def new_stat(prop_selected):
    ...

dec_new_stat = new_stat(26/31)
dec_new_stat

In [ ]:
ok.grade("q1_2_1");

**Question 1.2.2:** Define the function `new_simulate` that takes in an integer, `month`, and returns an array of 1000 simulated values of our new statistic under the null hypothesis.

*Hint: The table* `months` *should be very helpful!*

<!--
BEGIN QUESTION
name: q1_2_2
manual: true
-->
<!-- EXPORT TO PDF -->

In [28]:
months = lotto69.group("M")

def new_simulate(month):
    num_days_in_month = ...
    reps = ...
    
    test_stats = ...
    
    for i in np.arange(reps):
        one_sample = ...
        prop_selected_sample = ...
        one_new_stat = ...
        test_stats = ...
        
    ...

# DON'T CHANGE THE LINE
Table().with_column('Simulated Stats for January', new_simulate(1)).hist(0, bins=10) # Simulated stats for January

**Question 1.2.3:** Define the function `calc_p` that takes in an integer, `month` as an argument and returns the empirical p-value of the hypothesis test for that month.

*Hint: Add extra cells to test different pieces of your code!*

<!--
BEGIN QUESTION
name: q1_2_3
-->

In [29]:
def calc_p(month):
    simulated_test_stats = ...
    observed_num_drafted = ...
    num_days_in_month = ...
    observed_value = ...
    p_val = ...
    ...

january_p_value = calc_p(1)
january_p_value

In [ ]:
ok.grade("q1_2_3");

**Question 1.2.4:** Draw a bar chart of the p-values for each month. This might take some time to run.

*Hint: DO NOT TRY CALLING THE FUNCTION ON EVERY ROW IN THE LOTTO TABLE!*

<!--
BEGIN QUESTION
name: q1_2_4
manual: true
-->
<!-- EXPORT TO PDF -->

In [35]:
p_vals_per_month = ...
months_and_p_vals = months.with_column("p vals", p_vals_per_month)
months_and_p_vals.barh("M", "p vals")
# DON'T CHANGE THE FOLLOWING LINE OF CODE
plots.plot([0.05, 0.05], [0, 12], color="gold"); # Plots the cutoff of 5%

You should see 3 months with significant results.

**For the next 2 questions we are using a 5% p-value cutoff.**

**Question 1.2.5:** What is the probability that if the null hypothesis is true, we still reject it? This is also called the false positive rate or "alpha". Assign that value to `alpha`.

<!--
BEGIN QUESTION
name: q1_2_5
-->

In [36]:
alpha = ...
alpha

In [ ]:
ok.grade("q1_2_5");

**Question 1.2.6:** What is the probability that we get at least one false positive, if the null hypothesis is true, and we run 12 hypothesis tests (like we did above)? Assign that probability to `multiple_alpha`.

<!--
BEGIN QUESTION
name: q1_2_6
-->

In [39]:
multiple_alpha = ...
multiple_alpha

In [ ]:
ok.grade("q1_2_6");

Yikes, that number is pretty high. Luckily, statisticians have developed a few ways to deal with this problem.

**Question 1.2.7:** One way to deal with this issue is to use a lower p-value cutoff. To determine our new p-value cutoff, we can divide our original cutoff by the number of tests we are running. Calculate the probability of getting at least one false positive if we run 12 hypothesis tests and use this new adjusted p-value cutoff, assuming the null hypothesis is true in each case. Assign that value to `bonferroni`.

<!--
BEGIN QUESTION
name: q1_2_7
-->

In [43]:
adjusted_cutoff = ...
bonferroni = ...
bonferroni

In [ ]:
ok.grade("q1_2_7");

**Question 1.2.8:** Assign `num_sig` to the number of our hypothesis tests that were significant using our normal cutoff value. Assign `num_sig_adjusted` to the number of hypothesis tests that were significant using this new "adjusted" p-value cutoff.


<!--
BEGIN QUESTION
name: q1_2_8
-->

In [47]:
num_sig = ...
num_sig_adjusted = ...

print(f"Number of months significant: {num_sig}; Number of months significant after adjustment: {num_sig_adjusted}")

In [ ]:
ok.grade("q1_2_8");

**Question 1.2.9:** There are around 1300 students in Data 8 every semester, and around 500 in the summer. Presumably, each student will run the same hypothesis tests that you have ran in this experiment. Since this assignment is based on a very famous historical example, we can also assume forms of this exact hypothesis test have previously been run thousands if not millions of times. How do you think this huge number of hypothesis tests effects our conclusions, specifically related to our choice of p-value cutoff?


<!--
BEGIN QUESTION
name: q1_2_9
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

### Checkpoint (due Friday, 4/2 by 11:59pm PT)
#### Congratulations, you have reached the checkpoint! Run the submit cell below to generate the checkpoint submission. 

In [51]:
_ = ok.submit()

## Section 3: Applying the Central Limit Theorem

As part of a lesson on the Vietnam War, Jill plans on taking a survey of the birthdays of her 45 person class to determine who could have been drafted.

From the previous part, we know that 195 out of 366 of possible birthdates would be eligible. For every following question, we assume birth dates have an equal proportion in the population (this assumption is not true in reality), and we are only concered with the birthdate (ie we don't consider that the draft had limits on age or sex).

**Question 1.3.1:** Which of the following facts tell us that the Central Limit Theorem will dictate the distribution of the number of students with draft eligible birthdays? Assign `choices_clt` to an array that contain integers corresponding to valid statements.

1. The number of people in the class who could be drafted acts like a population sum.
2. Since the population the class is drawn from is *reasonably* large, the class acts like a sample drawn with replacement
3. 195/366 is fairly close to 50%
4. Birthdates in the population are not normally distributed
5. The class size is reasonably large

<!--
BEGIN QUESTION
name: q1_3_1
-->

In [52]:
choices_clt = ...

In [ ]:
ok.grade("q1_3_1");

**Question 1.3.2:** Plot a histogram of the distribution of the number of students with draft eligible birthdays. Use 10,000 simulated trials.

*Hint: Visualize an emprical distribution of the number of students with draft eligible birthdays in 10,000 simulated samples of size 45.*

<!--
BEGIN QUESTION
name: q1_3_2
-->

In [56]:
class_nums = ...
reps = ...
draft_proportions = ...
class_size = ...

...

# DON'T CHANGE THIS LINE
Table().with_column("Number Eligible", class_nums).hist(0, bins=np.arange(0.5,45.5,1))

Load the `birthdays` table in the following cell. The table contains the birthdays of Jill's class.

In [57]:
birthdays = Table().read_table("birthdays.csv")
birthdays.show(10)

**Question 1.3.3:** We need to use the lottery information from the `lotto` table to determine which of these birthdays are eligible. Create a new table called `birthdays_new` that has the `M`, `D` from `birthdays` and the `N69` column from `lotto`. The rows should correspond to the 45 rows in the `birthdays` table.

*Hint: Remember that you can join on multiple columns at the same time by passing **lists** as the column arguments. [Example](http://data8.org/datascience/_autosummary/datascience.tables.Table.join.html)*

<!--
BEGIN QUESTION
name: q1_3_3
-->

In [58]:
birthdays_new = ...
birthdays_new

In [ ]:
ok.grade("q1_3_3");

**Question 1.3.4:** Generate a 95% Confidence interval for the sample number of eligible students using 10,000 bootstrap resamples. Assign the bounds to `upper` and `lower`. Make sure to assign the 10,000 resample statistics to the array `resamples_num_eligible`.

*Hint: Do not redo the work you did in 1.3.3, otherwise your code will try to join two tables 10,000 times and run forever!*

<!--
BEGIN QUESTION
name: q1_3_4
-->

In [62]:
resamples_num_eligible = ...
reps = ...

for i in np.arange(reps):
    resample = ...
    num_eligible = ...
    resamples_num_eligible = ...

lower = ...
upper = ...

# DON'T CHANGE THESE LINES OF CODE
Table().with_column("Resample Values", resamples_num_eligible).hist(0, bins=np.arange(0.5,45.5,1))
plots.plot([lower,upper], [0,0], color="gold", lw=10); # Plots the CI
print(f"95% CI: [{lower}, {upper}])")

In [ ]:
ok.grade("q1_3_4");

**Question 1.3.5:** Steve did the same exercise with his own class. To see if the number eligible in his class is statistically significant or not, which of the two histograms should he compare his number to and why?


<!--
BEGIN QUESTION
name: q1_3_5
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

**Challenge:** Where do these birthdays come from? Post your guess on the Piazza thread for this section!

# Part 2: Lifetime Earnings

> "A central question in the debate over military manpower policy is whether veterans are adequately compensated for their service. The political process clearly reflects the desire to compensate veterans: since World War II, millions of veterans have enjoyed benefits for medical care, education and training, housing, insurance, and job placement. Recent legislation provides additional benefits for veterans of the Vietnam era. Yet, academic research has not shown 
conclusively that Vietnam (or other) veterans are worse off economically than nonveterans" (Angrist 1990)

Are veterans "adequately compensated"? Can we identify any significant differences in earnings between veterans and non-veterans? This was the central focus of the classic paper ["Lifetime Earnings and the Vietnam Era Draft Lottery: Evidence from Social Security Administrative Records"](https://economics.mit.edu/faculty/angrist/data1/data/angrist90) by [Joshua Angrist](https://en.wikipedia.org/wiki/Joshua_Angrist). In the following section, we will try to use methods we have learned, including hypothesis testing and parameter estimation to investigate this question.

## Section 1: The Data

>"The original CWHS data set does not contain information on date of birth. SSA programmers matched date of birth variables to the CWHS in a special extract created for this project. Lottery numbers were then matched to dates of birth, using tables published in the 1969-73 Semiannual Reports of the Director of Selective Service. The Internal Revenue Service limits disclosure of data collected for tax purposes. To adhere to these disclosure requirements, the SSA could release only aggregate data. The aggregate data set contains sample statistics for cells defined by year of earnings, year of birth, race, and five consecutive lottery numbers. Cell statistics include means, variances, fraction with earnings equal to the taxable maximum, fraction with earnings above the taxable maximum, fraction with zero earnings, and number of observations in each cell." (Angrist 1990)

Angrist collected data from the IRS (Internal Revenue Service-- the tax agency of the United States). The dataset in question is the CWHS: the Continuous Work History Sample, which is a sample of 1% of all eligible social security numbers. You can see what this table looks like by running the following cell.

In [68]:
cwhsb = Table.read_table("cwhsb.csv").drop(0)
cwhsb.show(5)

**Question 2.1.1:** Each row in `cwhsb` represents tax info of about 200 people grouped by range of birth date (interval represents a group of 5 days of the year). Why do you think the IRS would aggregate the data instead of releasing info for each person individually in the sample?

<!--
BEGIN QUESTION
name: q2_1_1
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

**Question 2.1.2:** How many people are represented in the `cwhsb` table? Assign that value to `total`. 

*Hint: The `vnu1` column tells you how many people each row corresponds to*
<!--
BEGIN QUESTION
name: q2_1_2
-->

In [69]:
total = ...
total

In [ ]:
ok.grade("q2_1_2");

Angrist applies a number of filters on the data to limit the data to just men of the appropriate ages who were eligible to be drafted in any of the four draft lotteries. After applying these filters, the new table represents only 1,238,734 men. After reproducing these filters, our data is still aggregated data.   

**Question 2.1.3:** Do you think using aggregated data or not-aggregated data would produce a more correct result for any hypothesis tests we perform?

<!--
BEGIN QUESTION
name: q2_1_3
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

For the rest of this section we are going to use artificially disaggregated data. Here's how Yanay disaggregated the data:

1. For each row in `cwshb`, he generated a corresponding number of rows in the new dataset with the same categorical variables (ie birth year, race, birthday interval etc) to represent each person aggregated in the `cwshb` row.
2. For the earnings for each new row, he randomly generated a value for earnings from a distribution that had the same mean and standard deviation as specified for that group in the `cwshb` table. Since the `cwshb` table has info about proportions of people in a group who had 0 earnings, he made sure that a corresponding number of people have an earnings of \$0 for that group in the new table.

For example, for this row from the `cwhsb` table,

![One Row](one_row_cwhsb.png)

Yanay generated 190 rows, 30 of which have earnings of 0 (0.163 * 190 == 30). The distribution of earnings in these rows looks like this:

![One Row Earnings](row_earnings.png)

The 190 rows all correspond to white men born in 1950 who fall into birthday interval 1 (birthday interval 1 contains 5 birth dates that were all eligible to be drafted). The mean of the earnings generated is 9150.06 and the standard deviation is 5562.51, which is close to what was stated in the `cwshb` table (the vmn1 and vsd1 columns contain this info).

The disaggregated rows are stored in the `earnings` table.

In [71]:
earnings = Table.read_table("disag.csv")
earnings.show(10)

As stated, Yanay randomly generated the (non zero) earnings by sampling from a distribution with the mean and SD specified, but with a histogram that looks something like this (with appropriate mean and sd):

![Curve Shape](shape_of_curve.png)

**Question 2.1.4:** Do you think using this method of disaggregating data is completely accurate? Can you think of any ways it might bias our data, alter our  analysis, or introduce any inaccuracies?

<!--
BEGIN QUESTION
name: q2_1_4
-->

*Write your answer here, replacing this text.*

## Section 2: Analysis

In this section we will test the hypothesis that draft eligibility (ie one's birthdate being chosen in the lottery) had a negative impact on one's earnings.

To make the data easier to use, in some parts we will use a random sample of 30000 rows from the earnings table instead of all 1.2 million.

In [72]:
np.random.seed(8)
sample = earnings.sample(30000, with_replacement=False)
sample.show(5), sample.hist(0, bins=np.arange(0, 80000, 10000));

In [73]:
sample.hist("earnings", group="eligible",bins=np.arange(0, 80000, 10000));
print("Table 1:")
sample.select("eligible", "earnings", "byr").group("eligible", np.mean).show()
print("Table 2:")
sample.select("byr", "earnings", "eligible").group("byr", np.mean).show()

**Question 2.2.1:** Table 1 seems to show us that lottery eligibile men had slightly higher earnings: based on Table 1 and Table 2, why do you think this is the case?

*Hint: remember that the `byr` column is birth year, i.e. 50 is 1950.*

<!--
BEGIN QUESTION
name: q2_2_1
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

Race also plays a factor in this dataset (see table 3 below). Unfortunately, the dataset is not very descriptive, and only has two options for race, white or non white. Non white is a very non-descriptive term for a huge, diverse group of people, and this may introduce a variety of confounding factors. So, for the purposes of this project we focus on the white population for which there exists clear historical records, and for which the Angrist paper investigated.

In [74]:
print("Table 3:")
sample.select("race", "earnings", "eligible").group("race", np.mean).show()

In [75]:
sample.where("byr", 50).where("race", "white").hist("earnings", group="eligible",bins=np.arange(0, 80000, 10000));
plots.title("Earnings for 1950 White men, by eligibility");
sample.where("byr", 50).where("race", "white").select("eligible", "earnings").group("eligible", np.mean).show()

Let's take another sample, stored in `white_50` that contains a random sample of 2000 white men born in 1950 from the `earnings` table. We've given you some important summary statistics for the table as well.

In [76]:
white_50 = earnings.where("byr", 50).where("race", "white").sample(2000, with_replacement=False).drop("byr", "race", "interval")
white_50.show(5);
white_50.group("eligible", np.mean).show();
white_50.group("eligible").show();

We want to check if earnings of the two groups, those eligible to be drafted and those not, come from the same distribution. Specifically, we want to check if those eligible to be drafted had lower earnings.

**Question 2.2.2:** What's our null hypothesis?
<!--
BEGIN QUESTION
name: q2_2_2
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

**Question 2.2.3:** What's our alternative hypothesis?

<!--
BEGIN QUESTION
name: q2_2_3
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

**Question 2.2.4:** Assign `choice_test_stat` to an integer corresponding to the number of the test statistic we could use to test this hypothesis.

1. The difference in mean earnings of white men eligible to be drafted in 1950 and those who were not eligible
2. The absolute difference in mean earnings of white men eligible to be drafted in 1950 and those who were not eligible
3. The mean earnings of white mean eligible to be drafted in 1950
4. The mean earnings of white mean eligible to be drafted in 1950 - 15168.5

<!--
BEGIN QUESTION
name: q2_2_4    
-->

In [77]:
choice_test_stat = ...

In [ ]:
ok.grade("q2_2_4");

**Question 2.2.5:** Define the function `test_stat` to return that test statistic for a given `tbl` like the white_50 table.

<!--
BEGIN QUESTION
name: q2_2_5
-->

In [81]:
def test_stat(tbl):
    grouped = ...
    ...

observed = ...
observed

In [ ]:
ok.grade("q2_2_5");

**Question 2.2.6:** Define the function `simulate_stat` that returns one value of the test statistic, simulated under the null hypothesis. 

<!--
BEGIN QUESTION
name: q2_2_6
-->

In [84]:
def simulate_stat():
    shuffled_column = ...
    shuffled_table = ...
    ...

simulate_stat()

In [ ]:
ok.grade("q2_2_6");

**Question 2.2.7:** Plot a histogram of 250 simulated statistics under the null hypothesis.

*Note: It may take some time for this cell to run*

<!--
BEGIN QUESTION
name: q2_2_7
manual: true
-->
<!-- EXPORT TO PDF -->

In [86]:
...

**Question 2.2.8:** Calculate the `p_value` of the test and assign `reject` to `True` if we should reject the null hypothesis using a 5% p-value cutoff or `False` otherwise.

<!--
BEGIN QUESTION
name: q2_2_8
-->

In [87]:
p_value = ...
reject = ...

print(f"Calculated p-value to be {p_value}; Should we reject? {reject}")

In [ ]:
ok.grade("q2_2_8");

## Section 3: A  New Statistic

Now let's test the same hypothesis using a new test statistic. This test statistic is called "Rank sum".

What is a rank? A rank is the position of a value when you sort the entire dataset in ascending order, starting at 1. 

For example, let's say we have the data 5, 8, 9, 7. The corresponding ranks are 1, 3, 4, 2, because the sorted data set is 5, 8, 9, 7.

We do have to worry about breaking ties for this dataset, because we have many zeros.

To break ties, we assign all values that are equal to the same rank. To do this, we must

1. Get a list of all the unique values
2. Sort that list in ascending order
3. Rank that list starting at 1
4. Assign those ranks back to the original data

For the data 6, 6, 7, 5, we get ranks 2, 2, 3, 1. For 0, 0, 0, 15.8, 12.6 we get 1, 1, 1, 3, 2.

The **rank sum** for a group within a dataset is equal to the sum of all the ranks of the values **within that group**.

We will use the rank sum for those eligible to be drafted as our new statistic.

*Note: Our alternative hypothesis is actually a bit different now, as we aren't concerned with the average, rather we are concerned with the rank sum, but this shouldn't affect your work.*

**Question 2.3.1:** Consider two samples:

|Sample 1| Sample 2|
|---------|------|
|1|3|
|2|4|
|10|6|

And their corresponding statistic values:

|Sample | Mean | Rank Sum |
|--------|-----|----------|
|Sample 1|4.333|9|
|Sample 2| 4.333| 12|

How might this Rank Sum statistic help us determine differences in numerical distributions in ways that the mean cannot? In your answer, make sure to specifically reference the provided samples.

<!--
BEGIN QUESTION
name: q2_3_1
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

**Question 2.3.2:** Which values of the rank sum statistic correspond to our alternative hypothesis that those who were eligible (white men born in 1950 who had their birthday chosen) had a lower income?

1. Larger Positive Values
2. Smaller Positive Values

Assign `which_direction` to either `1` or `2`.

<!--
BEGIN QUESTION
name: q2_3_2
-->

In [92]:
which_direction = ...

In [ ]:
ok.grade("q2_3_2");

**Question 2.3.3:** Implement the function `rank_sum_stat` that takes in a `tbl` like `white_50` and returns the rank sum for the eligible group's earnings.

To answer this question, define the function `tbl_with_ranks` that takes in any `tbl`, and a string `col` and returns a new table, sorted by `col` in ascending order, with a column called `Rank` that contains the ranks for the corresponding row, ranked by the values in `col`.

*Hint 1: ranks start at 1 and go up by 1 each value*

*Hint 2: make sure to rank all the data at the same time, and then just sum the ranks for the eligible group*

*Hint 3: remember that the group function combines equal values, including equal numerical values!*

*Hint 4: after you get the unique values, how could the join function be helpful?*

<!--
BEGIN QUESTION
name: q2_3_3
-->

In [95]:
def tbl_with_ranks(tbl, col):
    unique_values = ...
    sorted_unique_values = ...
    unique_ranks_arr = ...
    sorted_unique_with_ranks = ...
    
    original_table_with_unique_ranks = ...
    return original_table_with_unique_ranks.sort(col)

def rank_sum_test(tbl):
    tbl_and_ranks = tbl_with_ranks(tbl, "earnings")
    rank_sums = ...
    ...

observed_rank_sum = ...
observed_rank_sum

In [ ]:
ok.grade("q2_3_3");

**Question 2.3.4:** Define the function `simulate_rank_sum_stat` that returns one value of the rank sum test statistic, simulated under the null hypothesis.    

<!--
BEGIN QUESTION
name: q2_3_4
-->

In [101]:
def simulate_rank_sum_stat():
    shuffled_column = ...
    shuffled_table = ...
    ...

simulate_rank_sum_stat()

In [ ]:
ok.grade("q2_3_4");

**Question 2.3.5:** Plot a histogram of 100 simulated rank sum statistics under the null hypothesis.

*Note this may take some time to run*

<!--
BEGIN QUESTION
name: q2_3_5
manual: true
-->
<!-- EXPORT TO PDF -->

In [103]:
...

**Question 2.3.6:** Assign `p_value_rank` to the p-value of of the rank sum test and assign `reject_rank` to `True` if we should reject the null hypothesis using a 5% p-value cutoff or `False` otherwise.

<!--
BEGIN QUESTION
name: q2_3_6
-->

In [104]:
p_value_rank = ...
reject_rank = ...

print(f"Calculated p-value to be {p_value_rank}; Should we reject? {reject_rank}")

In [ ]:
ok.grade("q2_3_6");

**Question 2.3.7:** Angrist found that:

>"Estimates based on the draft lottery indicate that as much as ten years after their
discharge from service, white veterans who
served at the close of the Vietnam era earned
substantially less than nonveterans. The annual earnings loss to white veterans is on the
order of $3,500 current dollars, or roughly
15 percent of yearly wage and salary earnings in the early 1980s. In contrast, the estimated veteran effects for nonwhites are not
statistically significant"

Angrist used different methods (some of which we will learn about in the coming weeks). Using our methods, are we able to duplicate Angrist's findings? Do you think our findings invalidate Angrist's conclusions? Do Angrist's findings invalidate the methods we have taught you?

<!--
BEGIN QUESTION
name: q2_3_7
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

**You're finished!** Congratulations on mastering* hypothesis testing and bootstrapping. Time to submit.

## Submission

Once you're finished, select "Save and Checkpoint" in the File menu and then execute the `submit` cell below. The result will contain a link that you can use to check that your assignment has been submitted successfully. **IMPORTANT: Make sure to check that your submission is not empty. We cannot guarantee that the submission will be valid, so please check it now.** If you submit more than once before the deadline, we will only grade your final submission. If you mistakenly submit the wrong one, you can head to [okpy.org](https://okpy.org/) and flag the correct version. To do so, go to the website, click on this assignment, and find the version you would like to have graded. There should be an option to flag that submission for grading!

**NOTE:** The tests that are provided are not comprehensive and act as sanity checks (i.e. to make sure you answer is in the correct form, etc.). Passing the tests for a question does not mean that you answered the question correctly.

In [109]:
_ = ok.submit()

In [110]:
# For your convenience, you can run this cell to run all the tests at once!
import os
print("Running all tests...")
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q') and len(q) <= 10]
print("Finished running all tests.")